# Detailed Implementation Plan
1. **Adapt Knowledge from Task 1**: Advanced Analysis

- Objective: Build on initial time series methods by applying both univariate and multivariate models to capture dependencies and potential regime changes.

- Approach: Implement models like `VAR` and `Markov-Switching ARIMA` to handle market condition changes and dependencies among economic variables.

2. **Collect Economic, Technological, and Political Data
Data Source**: Use `wbdata` to pull indicators such as GDP, inflation, and exchange rates for major economies.

Integration: Add the new data to the original dataset and use merge_economic_data() to prepare for multivariate analysis.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd '/content/drive/MyDrive/Classroom/KifiyaAIM/week 10/Brent-Oil-Price-Analysis-and-Change-Point-Detection/notebooks'

/content/drive/MyDrive/Classroom/KifiyaAIM/week 10/Brent-Oil-Price-Analysis-and-Change-Point-Detection/notebooks


In [2]:
from IPython.display import clear_output
!pip install wbdata
clear_output()


In [3]:
 # Import necessary libraries along with our custom functions from /src
import sys
import os

# Get the current directory of this notebook
current_dir = os.path.dirname(os.path.abspath("__file__"))

# Construct the absolute path to the src directory
src_dir = os.path.join(current_dir, '..', 'src')

# Add the src directory to the system path
sys.path.append(src_dir)

import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
from data_preprocessing import merge_economic_data
from economic_data import fetch_economic_data
from advanced_models import fit_var, fit_markov_switching, fit_lstm
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Data Collection and Preparation

**Load Brent Oil Prices**: Load the oil_data from your preprocessed dataset.

**Fetch Economic Data**: Use fetch_economic_data() to collect relevant economic indicators like GDP, inflation, and exchange rates from the World Bank API.

**Merge Data**: Use merge_economic_data() to integrate economic indicators with oil prices for multivariate modeling.

In [13]:
oil_data = pd.read_csv("../data/BrentOilPrices.csv", parse_dates=["Date"])

# Fetch additional economic indicators
indicators = {
    "NY.GDP.MKTP.CD": "GDP (current US$)",
    "FP.CPI.TOTL": "Inflation, consumer prices (annual %)",
    "SL.UEM.TOTL.ZS": "Unemployment, total (% of total labor force)",
    "PA.NUS.FCRF": "Official exchange rate (LCU per US$, period average)"
}
# countries = ['USA', 'SAU', 'RUS', 'CAN', 'CHN']
economic_data = fetch_economic_data(indicators, start_date='1987-05-20', end_date='2022-09-30')


<ipython-input-13-6e019e75fea7>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  oil_data = pd.read_csv("../data/BrentOilPrices.csv", parse_dates=["Date"])


In [19]:
oil_data.shape

(9011, 2)

In [14]:
oil_data.head()

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63


In [15]:
economic_data.head()

GDP (current US$)  Inflation, consumer prices (annual %)  \
country date                                                             
Canada  2022       2.161483e+12                             129.858329   
        2021       2.007472e+12                             121.587006   
        2020       1.655685e+12                             117.594448   
        2019       1.743725e+12                             116.757298   
        2018       1.725329e+12                             114.524900   

              Unemployment, total (% of total labor force)  \
country date                                                 
Canada  2022                                         5.280   
        2021                                         7.527   
        2020                                         9.657   
        2019                                         5.690   
        2018                                         5.837   

              Official exchange rate (LCU per US$, period average)  
country date                                                        
Canada  2022                                           1.301555     
        2021                                           1.253877     
        2020                                           1.341153     
        2019                                           1.326793     
        2018                                           1.295818

In [16]:
# Reset index and drop the country column to avoid multi-level issues
economic_data = economic_data.reset_index()

# # Rename the 'date' column to 'Date' for consistency with brentoil_data
# economic_data.rename(columns={"date": "Date"}, inplace=True)

# # Convert the 'Date' column to datetime format
# economic_data['Date'] = pd.to_datetime(economic_data['Date'])

In [17]:
economic_data.tail()

,country,date,GDP (current US$),"Inflation, consumer prices (annual %)","Unemployment, total (% of total labor force)","Official exchange rate (LCU per US$, period average)"
175,United States,1991,6.158129e+12,62.457341,6.8,1.0
176,United States,1990,5.963144e+12,59.919760,NaN,1.0
177,United States,1989,5.641580e+12,56.850970,NaN,1.0
178,United States,1988,5.236438e+12,54.233135,NaN,1.0
179,United States,1987,4.855215e+12,52.108294,NaN,1.0


In [ ]:
# Update the columns of the oil data to merge with the economic_data
# Merge oil prices and economic data
merged_data = merge_economic_data(oil_data, economic_data)